In [1]:
print("ok all")

ok all


In [2]:
import os,sys
projject_root = os.path.abspath(os.path.join(os.getcwd(),".."))
sys.path.append(projject_root)

print(projject_root)

d:\Sunny_savitha\auto_rec\automated-research-report-generation


In [3]:
from research_and_analysts.utils.model_loader import ModelLoader


In [4]:
model_loader = ModelLoader()

{"timestamp": "2025-10-17T10:19:55.204165Z", "level": "info", "event": "GOOGLE_API_KEY loaded from environment"}
{"timestamp": "2025-10-17T10:19:55.206331Z", "level": "info", "event": "GROQ_API_KEY loaded from environment"}
{"timestamp": "2025-10-17T10:19:55.207945Z", "level": "info", "event": "ASTRA_DB_API_ENDPOINT loaded from environment"}
{"timestamp": "2025-10-17T10:19:55.209526Z", "level": "info", "event": "ASTRA_DB_APPLICATION_TOKEN loaded from environment"}
{"timestamp": "2025-10-17T10:19:55.211169Z", "level": "info", "event": "ASTRA_DB_KEYSPACE loaded from environment"}
{"config_keys": ["astra_db", "embedding_model", "retriever", "llm"], "timestamp": "2025-10-17T10:19:55.228053Z", "level": "info", "event": "YAML config loaded"}


In [5]:
llm = model_loader.load_llm()

{"provider": "google", "model": "gemini-2.0-flash", "timestamp": "2025-10-17T10:19:55.241055Z", "level": "info", "event": "Loading LLM"}


In [6]:
llm.invoke("Hi").content

'Hi there! How can I help you today?'

In [7]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

In [8]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

In [9]:
class Analyst(BaseModel):
    name:str = Field(description="Name of analyst.")
    role:str = Field(description="Role of analyst in context of topic")
    affiliation:str = Field(description="Primary affiliation of the analyst")
    description:str = Field(description="Description of the analyst focus, concerns and motives.")

    @property
    def persona(self)->str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"
    

In [10]:
analyst = Analyst(
    name="Munavar",
    role="gen_ai eng",
    affiliation="AI Research Lab",
    description="I am genai eng"
)

In [11]:
analyst.name

'Munavar'

In [12]:
print(analyst.persona)

Name: Munavar
Role: gen_ai eng
Affiliation: AI Research Lab
Description: I am genai eng



In [13]:
class Perspective(BaseModel):
    analysts: List[Analyst] = Field(description="Comprehensive list of analyst with theor roles and affiliations.")

In [14]:
class GenerativeAnalystState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback:str
    analysts: List[Analyst]

In [15]:
GenerativeAnalystState(
    topic= "finance",
    max_analysts= 5,
    human_analyst_feedback="Give the real info"
)

{'topic': 'finance',
 'max_analysts': 5,
 'human_analyst_feedback': 'Give the real info'}

In [16]:
analyst_instructions="""You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:

1. First, review the research topic:
{topic}
        
2. Examine any editorial feedback that has been optionally provided to guide creation of the analysts: 
        
{human_analyst_feedback}
    
3. Determine the most interesting themes based upon documents and / or feedback above.
                    
4. Pick the top {max_analysts} themes.

5. Assign one analyst to each theme."""

In [17]:
def create_analyst(state:GenerativeAnalystState):
    topic = state["topic"]
    max_analysts = state["max_analysts"]
    human_analyst_feedback = state.get("human_analyst_feedback","")

    structured_llm = llm.with_structured_output(Perspective)
    system_message = analyst_instructions.format(
        topic = topic,
        max_analysts=max_analysts,
        human_analyst_feedback = human_analyst_feedback
    )
    analysts = structured_llm.invoke([SystemMessage(content=system_message)]+[HumanMessage(content="Generate the set of analyst. ")])
    return {"analysts": analysts.analysts}

In [18]:
create_analyst(
 {'topic': 'finance',
 'max_analysts': 5,
 'human_analyst_feedback': 'Give the real info'})

{'analysts': [Analyst(name='Ava Sterling', role='Quantitative Analyst', affiliation='Hedge Fund X', description='Focuses on statistical modeling and algorithmic trading strategies to maximize returns and manage risk. Concerned with market volatility and data accuracy.'),
  Analyst(name='Ben Carter', role='Financial Regulation Specialist', affiliation='Regulatory Compliance Agency Y', description='Monitors financial institutions for compliance with regulations, aiming to prevent fraud and maintain market stability. Motivated by ethical investing and consumer protection.'),
  Analyst(name='Chloe Davis', role='Investment Strategist', affiliation='Global Investment Bank Z', description='Develops investment strategies based on macroeconomic trends and company performance. Aims to identify growth opportunities and manage portfolio risk. Driven by long-term value creation.'),
  Analyst(name='David Evans', role='Corporate Finance Advisor', affiliation='Mergers & Acquisitions Firm A', descripti

In [19]:
def human_feedback(state):
    pass

In [20]:
def should_continue(state):
    human_analyst_feedback = state.get("human_analyst_feedback", None)
    if human_analyst_feedback:
        return "create_analyst"

In [21]:
builder = StateGraph(GenerativeAnalystState)

In [22]:
builder.add_node("create_analyst", create_analyst)
builder.add_node("human_feedback", human_feedback)

In [23]:
builder.add_edge(START, create_analyst)
builder.add_edge("create_analyst","human_feedback")
builder.add_conditional_edges("human_feedback",
                         should_continue,
                         ["create_analyst",
                          END])

In [24]:
memory = MemorySaver()

In [25]:
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer = memory)

ValueError: Found edge ending at unknown node `<function create_analyst at 0x0000021B7CB631A0>`

In [ ]:
display(Image(graph.get_graph(xray=1).draw_mermaid_png))